In [1]:
import wfdb as wf
from wfdb import processing

import neurokit2 as ECG
import librosa as mi


import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, ifft
import numpy as np
import os

In [2]:
def Feature_CC(segment):
    try:
        win = signal.windows.hamming(len(segment))
        win_segment = np.multiply(segment, win)

        fft_c = fft(win_segment,len(win_segment))
        log_fft_c = np.log(np.abs(fft_c))
        cc= ifft(log_fft_c)

        return np.real(cc[:12])
    except IndexError:
        print("Index_error")

def Feature_Entropy(segment):


    counts, _ = np.histogram(segment, bins=5)
    P_x = counts/sum(counts)

    entropy = sum(np.multiply(P_x,np.log2(P_x)))

    return entropy
        


def Feature_ZCR(segment):
    return np.sum(mi.zero_crossings(segment).astype(float))

def ConvertArray2String(B):
    Feature = ""
    for i in B:
        Feature = Feature + ',' +str(i)
    return Feature


In [3]:
os.chdir('..')
dir = os.getcwd()

In [4]:
os.chdir(dir)
os.chdir(dir + '/DATASET/MIT-BIH')
IDs = os.listdir()
print(IDs)

# Channel number
N = 0

f = open(dir + '/Features' +'/'+ f"Feature_MIT_BIH_Channel_{N}.csv", "a")
f.write("ID,CC_1,CC_2,CC_3,CC_4,CC_5,CC_6,CC_7,CC_8,CC_9,CC_10,CC_11,CC_12,ENTROPY,ZCR,TARGET\n")

for ID in IDs:
    print(ID)
    os.chdir(  os.getcwd()+ '/'+ ID)
    Audio = os.listdir()
    Audio_list = [k for k in Audio if '.dat' in k]



    for A in Audio_list:
        
        print(A)
        signal_array, fields = wf.rdsamp( os.getcwd() +  "/" + A.replace(".dat", ""))
        sig = signal_array[:,N]

        # For single channel
        sos = signal.butter(6, [2, 50], btype='bandpass', fs=fields['fs'], analog= False)
        filtered = signal.filtfilt(sos[0], sos[1], sig)

        try:
            _, rpeaks = ECG.ecg_peaks(filtered, sampling_rate=fields['fs'])
            _, waves_peak = ECG.ecg_delineate(filtered, rpeaks, sampling_rate=1000, method="peak")
            t_peaks_ind = waves_peak['ECG_T_Peaks']
        except IndexError:
            print('Skipping' + A)
            break
        except TypeError:
            print('Skipping' + A)
            break
        except ValueError:
            print('Skipping' + A)
            break
        

        for n in range(len(t_peaks_ind)-1):

            try:
                segment = filtered[int(t_peaks_ind[n]):int(t_peaks_ind[n+1])]
                CC = Feature_CC(segment)
                Entropy = Feature_Entropy(segment) 
                ZCR = Feature_ZCR(segment)

                feature_string = ID + '_' + A.replace('.dat','') +'_' + str(n) + ConvertArray2String(CC) + ',' + str(Entropy) + ',' +  str(ZCR) + ',' + A.replace('.dat','').replace('x_','') + '\n'
                f.write(feature_string)
            except ValueError:
                print('Skipping a segment')


    os.chdir(dir + '/DATASET/MIT-BIH')

f.close()

['Records_2', 'Records_1']
Records_2
x_123.dat
Skipping a segment
x_222.dat


/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: divide by zero encountered in log2
  entropy = sum(np.multiply(P_x,np.log2(P_x)))
/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: invalid value encountered in multiply
  entropy = sum(np.multiply(P_x,np.log2(P_x)))


x_233.dat
x_114.dat
x_231.dat


/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: divide by zero encountered in log2
  entropy = sum(np.multiply(P_x,np.log2(P_x)))
/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: invalid value encountered in multiply
  entropy = sum(np.multiply(P_x,np.log2(P_x)))


x_230.dat
x_122.dat
x_232.dat


...

/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: divide by zero encountered in log2
  entropy = sum(np.multiply(P_x,np.log2(P_x)))
/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: invalid value encountered in multiply
  entropy = sum(np.multiply(P_x,np.log2(P_x)))


123.dat
231.dat


/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: divide by zero encountered in log2
  entropy = sum(np.multiply(P_x,np.log2(P_x)))
/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: invalid value encountered in multiply
  entropy = sum(np.multiply(P_x,np.log2(P_x)))


109.dat
232.dat


/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: divide by zero encountered in log2
  entropy = sum(np.multiply(P_x,np.log2(P_x)))
/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: invalid value encountered in multiply
  entropy = sum(np.multiply(P_x,np.log2(P_x)))


In [5]:
os.chdir(dir)
os.chdir(dir + '/DATASET/MIT-BIH')
IDs = os.listdir()
print(IDs)

# Channel number
N = 1

f = open(dir + '/Features' +'/'+ f"Feature_MIT_BIH_Channel_{N}.csv", "a")
f.write("ID,CC_1,CC_2,CC_3,CC_4,CC_5,CC_6,CC_7,CC_8,CC_9,CC_10,CC_11,CC_12,ENTROPY,ZCR,TARGET\n")

for ID in IDs:
    print(ID)
    os.chdir(  os.getcwd()+ '/'+ ID)
    Audio = os.listdir()
    Audio_list = [k for k in Audio if '.dat' in k]



    for A in Audio_list:
        
        print(A)
        signal_array, fields = wf.rdsamp( os.getcwd() +  "/" + A.replace(".dat", ""))
        sig = signal_array[:,N]

        # For single channel
        sos = signal.butter(6, [2, 50], btype='bandpass', fs=fields['fs'], analog= False)
        filtered = signal.filtfilt(sos[0], sos[1], sig)

        try:
            _, rpeaks = ECG.ecg_peaks(filtered, sampling_rate=fields['fs'])
            _, waves_peak = ECG.ecg_delineate(filtered, rpeaks, sampling_rate=1000, method="peak")
            t_peaks_ind = waves_peak['ECG_T_Peaks']
        except IndexError:
            print('Skipping' + A)
            break
        except TypeError:
            print('Skipping' + A)
            break
        except ValueError:
            print('Skipping' + A)
            break
        

        for n in range(len(t_peaks_ind)-1):

            try:
                segment = filtered[int(t_peaks_ind[n]):int(t_peaks_ind[n+1])]
                CC = Feature_CC(segment)
                Entropy = Feature_Entropy(segment) 
                ZCR = Feature_ZCR(segment)

                feature_string = ID + '_' + A.replace('.dat','') +'_' + str(n) + ConvertArray2String(CC) + ',' + str(Entropy) + ',' +  str(ZCR) + ',' + A.replace('.dat','').replace('x_','') + '\n'
                f.write(feature_string)
            except ValueError:
                print('Skipping a segment')

    os.chdir(dir + '/DATASET/MIT-BIH')

f.close()

['Records_2', 'Records_1']
Records_2
x_123.dat
x_222.dat
x_233.dat
x_114.dat
x_231.dat
x_230.dat
x_122.dat
x_232.dat
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
x_111.dat
Skipping a segment
x_223.dat
x_234.dat
x_121.dat
Skipping a segment
x_109.dat
x_113.dat
x_108.dat
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
x_124.dat
x_115.dat
x_221.dat
x_220.dat
x_116.dat
x_117.dat
x_228.dat
x_112.dat
Records_1
203.dat
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
100.dat


/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: divide by zero encountered in log2
  entropy = sum(np.multiply(P_x,np.log2(P_x)))
/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: invalid value encountered in multiply
  entropy = sum(np.multiply(P_x,np.log2(P_x)))


118.dat
223.dat
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
200.dat


/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: divide by zero encountered in log2
  entropy = sum(np.multiply(P_x,np.log2(P_x)))
/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: invalid value encountered in multiply
  entropy = sum(np.multiply(P_x,np.log2(P_x)))


Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
202.dat
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment


...

Skipping a segment
Skipping a segment
210.dat
201.dat
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment


/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: divide by zero encountered in log2
  entropy = sum(np.multiply(P_x,np.log2(P_x)))
/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: invalid value encountered in multiply
  entropy = sum(np.multiply(P_x,np.log2(P_x)))


Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
122.dat
114.dat


/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: divide by zero encountered in log2
  entropy = sum(np.multiply(P_x,np.log2(P_x)))
/tmp/ipykernel_3893/427523320.py:20: RuntimeWarning: invalid value encountered in multiply
  entropy = sum(np.multiply(P_x,np.log2(P_x)))


230.dat
217.dat
Skipping a segment
102.dat
221.dat
Skipping a segment
Skipping a segment
113.dat
220.dat
Skipping a segment
123.dat
231.dat
109.dat
232.dat
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment
Skipping a segment


In [6]:
for n in range(0,10):
    try:
        Print
    except NameError:
        print("Error")
        break

    print(n)

Error
